### Qiime 2 

This walk through with the mock data is based off the tutorial shown [here](https://docs.qiime2.org/2018.2/tutorials/moving-pictures/). Additional resources and help can be found at the Qiime 2 [webpage](https://qiime2.org/) and the Qiime 2 [Forum](https://forum.qiime2.org/). Additional instructions on the installation of Qiime can be found [here](https://docs.qiime2.org/2018.2/install/)

In [ ]:
## Install with conda
#!wget https://data.qiime2.org/distro/core/qiime2-2018.2-py35-osx-conda.yml
#!conda env create -n qiime2-2018.2 --file qiime2-2018.2-py35-osx-conda.yml

## Optional Cleanup 
#!rm qiime2-2018.2-py35-osx-conda.yml

### Activate the Environment
The following chunk tests if you have properly installed and activated the qiime2 environment. If there is an error, exit the jupyter notebook (ctrl-c) on the terminal and activate the qiime environment with **source activate qiime2-2018.2**

In [18]:
!qiime --help

### Creating a Manifest File
A manifest file is needed to point Qiime functions to the path of fastq.gz files. NOTE: The files must be in gzipped form. Below is an example example of a manifest file with three columns: sample-id, absolute-filepath, and direction

In [34]:
import csv
with open("./Qiime_demo/manifest_file.csv", 'r') as f:
    data = list(csv.reader(f))

In [32]:
data

[['sample-id', 'absolute-filepath', 'direction'],
 ['sample-1',
  '/Users/stronglab2/Documents/Microbiome_Demo/data/trimmed_mock_R1.fastq.gz',
  'forward'],
 ['sample-1',
  '/Users/stronglab2/Documents/Microbiome_Demo/data/trimmed_mock_R2.fastq.gz',
  'reverse']]

### Quality Filtering Reads and Demultiplexing Reads
Demultiplexing and filtering reads base on quality score, more information about filtering reads and importing data can be found [here](https://docs.qiime2.org/2018.2/tutorials/importing/)

In [8]:
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path ./Qiime_demo/manifest_file.csv \
    --output-path ./Qiime_demo/paired-end-demux.qza \
    --source-format PairedEndFastqManifestPhred33

### Running Dada2 Denoising Algorithm
Trimming both forward and reverse reads to 230 base pairs. This runs the core denoising algorithm of dada2

In [11]:
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs ./Qiime_demo/paired-end-demux.qza \
    --p-trunc-len-f 230 \
    --p-trunc-len-r 230 \
    --o-representative-sequences ./Qiime_demo/rep-seqs.qza \
    --o-table ./Qiime_demo/table.qza

Saved FeatureTable[Frequency] to: table-dada2.qza
Saved FeatureData[Sequence] to: rep-seqs-dada2.qza


### Summarize Table from Dada2 Output

In [2]:
!qiime feature-table summarize \
    --i-table ./Qiime_demo/table.qza \
    --o-visualization ./Qiime_demo/table.qzv

Saved Visualization to: table.qzv


In [19]:
## View Table {Note: Interupt Kernal needed when run from Jupyter}
#!qiime tools view ./Qiime_demo/table.qzv

### Taxonomic Assignment
Qiime2 uses GreenGenes 13-8 clustered at 99% and a naive bayes classifier to assign reads to taxa. 

In [ ]:
## Download classifier
#!wget -O "./Qiime_demo/gg-13-8-99-515-806-nb-classifier.qza" "https://data.qiime2.org/2018.2/common/gg-13-8-99-515-806-nb-classifier.qza"

In [24]:
!qiime feature-classifier classify-sklearn \
    --i-classifier ./Qiime_demo/gg-13-8-99-515-806-nb-classifier.qza \
    --i-reads ./Qiime_demo/rep-seqs.qza \
    --o-classification ./Qiime_demo/taxonomy.qza

Saved FeatureData[Taxonomy] to: Qiime_demo/taxonomy.qza


In [25]:
!qiime metadata tabulate \
    --m-input-file ./Qiime_demo/taxonomy.qza \
    --o-visualization ./Qiime_demo/taxonomy.qzv

Saved Visualization to: Qiime_demo/taxonomy.qzv


In [36]:
## View Table {Note: Interupt Kernal needed when run from Jupyter}
#!qiime tools view ./Qiime_demo/taxonomy.qzv

#### Create Taxa Bar Plots

In [28]:
!qiime taxa barplot \
    --i-table ./Qiime_demo/table.qza \
    --i-taxonomy ./Qiime_demo/taxonomy.qza \
    --m-metadata-file ./Qiime_demo/simple_metadata.tsv \
    --o-visualization ./Qiime_demo/taxa-bar-plots.qzv

Saved Visualization to: Qiime_demo/taxa-bar-plots.qzv


In [35]:
## View Table {Note: Interupt Kernal needed when run from Jupyter}
#!qiime tools view ./Qiime_demo/taxa-bar-plots.qzv

### Additional Tutorials

#### Qiime within Python

In [9]:
from qiime2 import Artifact
from qiime2.plugins import feature_table
unrarefied_table = Artifact.load('./Qiime_demo/table.qza')
rarefy_result = feature_table.methods.rarefy(table=unrarefied_table, sampling_depth=10000)
rarefied_table = rarefy_result.rarefied_table

In [10]:
import biom
biom_table = rarefied_table.view(biom.Table)
print(biom_table.head())

# Constructed from biom file
#OTU ID	sample-1
9459cc87372e0014ceb236babf6e1693	389.0
76e797395c8bc004db520db5b47511a4	292.0
731b491457472e36422564468f57fa8c	129.0
3a8a7409bdc33da14a01ec3e3313b9a7	86.0
e2f910415582cce27eded24e82d682e4	39.0


### Create Tree for Diversity Analyses

#### Generate Tree for Phylogenetic Analysis

In [11]:
!qiime alignment mafft \
    --i-sequences ./Qiime_demo/rep-seqs.qza \
    --o-alignment ./Qiime_demo/aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza


#### Mask highly variable positions
Highly variable positions add noise to the resulting phylogenetic tree

In [12]:
!qiime alignment mask \
    --i-alignment ./Qiime_demo/aligned-rep-seqs.qza \
    --o-masked-alignment ./Qiime_demo/masked-aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza


#### Make unrooted and rooted trees
FastTree generates a tree alignment from the masked rep set and then is rooted using midpoint rooting to place the root at the midpoint of the longest tip-to-tip distance in the unrooted tree

In [13]:
## Unrooted tree
!qiime phylogeny fasttree \
    --i-alignment ./Qiime_demo/masked-aligned-rep-seqs.qza \
    --o-tree ./Qiime_demo/unrooted-tree.qza

Saved Phylogeny[Unrooted] to: unrooted-tree.qza


In [14]:
## Rooted Tree 
!qiime phylogeny midpoint-root \
    --i-tree ./Qiime_demo/unrooted-tree.qza \
    --o-rooted-tree ./Qiime_demo/rooted-tree.qza

Saved Phylogeny[Rooted] to: rooted-tree.qza


### Diversity Analysis

In [ ]:
## NOTE: Given a sample size of one this script does not function
!qiime diversity core-metrics-phylogenetic \
    --i-phylogeny ./Qiime_demo/rooted-tree.qza \
    --i-table ./Qiime_demo/table.qza \
    --p-sampling-depth 13000 \
    --m-metadata-file ./Qiime_demo/simple_metadata.tsv \
    --output-dir ./Qiime_demo/core-metrics-results